In [0]:
import numpy
import torch 
import torchvision 
import os
import matplotlib.pyplot as plt
import random 
import pdb
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, sampler
import imageio
import PIL
from IPython import display
from torchvision import datasets
import torch.nn as nn




In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!unzip "/content/gdrive/My Drive/Colab Notebooks/Data/Cassava_t_v_t.zip"

In [0]:
%matplotlib inline

def show(img):
    """Show PyTorch tensor img as an image in matplotlib."""
    npimg = img.cpu().detach().numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)), interpolation='nearest')
    plt.grid(False)
    plt.gca().axis('off')

def display_thumb(img):
  display.display(transforms.Resize(128)(img))

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## **Prepare the dataset** (balancing, spiliting, data augmentation, ..etc)

In [0]:
# Load Cassava dataset
traindata_folder_path = '/content/ammi-2020-convnets/train/train'
valdata_folder_path = '/content/ammi-2020-convnets/train/validation'
testdata_folder_path = '/content/ammi-2020-convnets/test/test'


# load the extra data for self-supervie (Autoencodrer)..
extra_folder_path = '/content/ammi-2020-convnets/extraimages'


In [0]:
# compute the mean and standard deviation of the all images (train, val, test)

In [33]:
# torch_example = torch.randn((100, 3, 50, 50))
# torch_example_mean = torch_example.mean(axis=(0,2,3))
# torch_example_mean
# # torch_example_mean2 = torch_example.mean(axis=(2,3))
# print('per batch = {0}'.format(torch_example_mean2.shape))
# torch_example_mean2 = torch_example_mean2.mean(axis=(0))
# print('per batch_and h and w = {0}'.format(torch_example_mean2.shape))


tensor([ 0.0001, -0.0008, -0.0002])

In [0]:
tf = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])
cassava_train = datasets.ImageFolder(traindata_folder_path, transform=tf)
trainloader = torch.utils.data.DataLoader(cassava_train, batch_size=700, shuffle=True, num_workers=8)

In [0]:
# compute the mean and standard deviation of the images ...
run_mean = torch.zeros(3)
for i, (batch,_) in enumerate(trainloader):
  curr_mean = batch.mean(axis=(0,2,3))
  run_mean = ((run_mean*i) + curr_mean)/(i+1) 
  


In [64]:
run_mean 

tensor([0.4479, 0.4963, 0.3217])

In [71]:

for imgs,_ in trainloader:
    print(imgs.std((0,2,3)))
    break

tensor([0.2074, 0.2087, 0.1896])


In [0]:
######################### Transformations of images #################################################


inference_transform = transforms.Compose([
    transforms.Resize((500, 500)),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize([0.4479, 0.4963, 0.3217], [0.2074, 0.2087, 0.1896])
])

# For visualization purposes we'll create a separate transform that operates in image space.
inference_transform_show = transforms.Compose([
    transforms.Resize(500),
    transforms.CenterCrop(256),
])


########################################   Transform and put the data into Dataset object #########################################################################

transformed_cassava_trian = datasets.ImageFolder(traindata_folder_path, transform=inference_transform)
transformed_cassava_validation = datasets.ImageFolder( valdata_folder_path, transform=inference_transform) 
transformed_cassava_test = datasets.ImageFolder(testdata_folder_path,  transform=inference_transform)

##########################################  Load the data into DataLoader #########################################################
transformed_cassava_trainloader = torch.utils.data.DataLoader(transformed_cassava_trian, batch_size=32, shuffle=True, num_workers=8)
transformed_cassava_valloader = torch.utils.data.DataLoader(transformed_cassava_validation, batch_size=32, shuffle=False, num_workers=8)
transformed_cassava_testloader = torch.utils.data.DataLoader(transformed_cassava_test, batch_size=32, shuffle=False, num_workers=8)




########## Extra data set loading ##########################################################################

transformed_cassava_extra = datasets.ImageFolder(extra_folder_path, transform=inference_transform)
transformed_cassava_extraloader = torch.utils.data.DataLoader(transformed_cassava_extra , batch_size=32, shuffle=True, num_workers=8)



In [0]:
test_loader.dataset.classes

['0']

In [0]:
len(train_loader.dataset)

5656

## **Basic function for training & testing**

In [0]:
def accuracy(output, target, topk=(1,)):
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [0]:
def train (model, criterion, dataloader, optimizer, num_epochs):
    
    # put the model in train mode
    model.train()
    
    # move the model to GPU
    model.to(device)
    


    losses = []
    for epoch in range(num_epochs):
      
      print("Epoch {}".format(epoch))
      avg_loss = 0
      correct = 0

      for batch_index, (batch, target) in enumerate(dataloader):
          batch = batch.to(device)
          target = target.to(device)
          
          optimizer.zero_grad()

          # Forward step 
          output = model(batch)
          loss = criterion(output.to(device), target)
          
          # Backward step 
          loss.backward()
          #gradient step
          optimizer.step()
          

          # compute the accuracy
          prediction = output.argmax(dim=1, keepdim=True)
          correct += prediction.cpu().eq(target.cpu().view_as(prediction)).sum().item()

          # avarge of loss
          avg_loss += loss.item()
          # print("Loss : {}".format(loss.item()))

      # losses.append(avg_loss/len(batch))    
      percent = 100. * correct / len(dataloader.dataset)
      # Print out progress the end of epoch.
      print('Train Epoch: {} \tLoss: {:.6f} \t Accuracy: {:.2f}%'.format(
              epoch, avg_loss/len(batch), percent)
        )

def test (model, criterion, dataloader):
    
    model.eval()
    correct = 0
    # avg_loss = 0
    with torch.no_grad():
        
        for data, target in dataloader:
             
            data = data.to(device)
            target = target.to(device)
            
            output = model(data).to(device)
            loss = criterion(output, target) 
            
            prediction = output.argmax(dim=1, keepdim=True) 
            correct += prediction.cpu().eq(target.view_as(prediction)).sum().item()
            
            # avg_loss += loss.item() 
    percent = 100. * correct / len(dataloader.dataset)
    print(f'Accuracy: {correct}/{len(dataloader.dataset)} ({percent:.0f}%)')
   
    return percent  
    
    
    
        
        
            

## **Model Architecture (ResNet)** 

In [0]:
# define the per-trained model...
full_resnet_model = torchvision.models.resnet18(pretrained=True)
# full_resnet_model

In [0]:
class ModelResNet(nn.Module):

  def __init__(self):
    super(ModelResNet, self).__init__()
    self.res_mod = nn.Sequential(*list(full_resnet_model.children())[:-1], nn.Linear(512, 5))


  def forward(x):
      x = self.res_mod(x)
      return x

In [0]:
# Model Architectuer..
model = ModelResNet()
model

## **Run the Model**  (checkpoint save and load, optimizer,...)

In [0]:
# Create checkpoint ...

model_save_name = 'classifier.pt'
path = "/content/gdrive/My Drive/{{model_save_name}"
torch.save(model.state_dict(), path)

In [0]:
# load the checkpoint...
model = model.load_state_dict(torch.load("/content/gdrive/My Drive/{model_save_name}"))
# model

In [0]:
if device == 'cuda':
    model = model.cuda()
    
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
train(model, nn.CrossEntropyLoss(), transformed_imagenet_loader, optimizer, 10)

Epoch 0


KeyboardInterrupt: ignored

In [0]:
transformed_imaf

In [0]:
linet = nn.Sequential( nn.Conv2d(3, 8, 3, stride=2),
nn.ReLU(),
nn.Conv2d(8, 128, 3, stride=2),
nn.ReLU(),
nn.ConvTranspose2d(128, 16, 3, stride=2))

In [0]:
x = torch.randn((4, 3, 27, 27))

In [0]:

linet(x).size()

torch.Size([4, 16, 13, 13])